In [1]:
!git lfs install

Git LFS initialized.


In [2]:
!git clone https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2 


Cloning into 'all-MiniLM-L6-v2'...
remote: Enumerating objects: 46, done.
remote: Total 46 (delta 0), reused 0 (delta 0), pack-reused 46
Unpacking objects: 100% (46/46), 311.33 KiB | 622.00 KiB/s, done.
Filtering content: 100% (3/3), 260.15 MiB | 10.52 MiB/s, done.


In [3]:
!pwd

/home/asyin/Downloads


In [4]:
!git clone https://huggingface.co/google/flan-t5-large


Cloning into 'flan-t5-large'...


remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 110 (delta 0), reused 0 (delta 0), pack-reused 107
Receiving objects: 100% (110/110), 635.37 KiB | 1.48 MiB/s, done.
Resolving deltas: 100% (58/58), done.
^C
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'


Exiting because of "interrupt" signal.


In [5]:
!pip install langchain
!pip install torch
!pip install transformers
!pip install faiss-cpu
!pip install pypdf
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: gpg 1.16.0-unknown has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of gpg or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that the

RAG PROCESS

In [6]:
from langchain.document_loaders import PyPDFLoader
pdfLoader = PyPDFLoader("/home/asyin/Downloads/sop_cohere.pdf")
documents = pdfLoader.load()


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(documents)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
modelPath = "/home/asyin/llama/all-MiniLM-L6-v2"
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':False}
embeddings = HuggingFaceEmbeddings(
  model_name = modelPath,  
  model_kwargs = model_kwargs,
  encode_kwargs=encode_kwargs
)

2023-10-10 10:32:46.504197: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 10:32:47.123249: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [29]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)
question = "What are her interests"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)


Statement of Purpose                                      Nevasini S asikumar                                          September , 2023  
 
 I am writing to express my profound enthusiasm for the Cohere for AI Scholars Program, a unique 
opportunity that impeccably aligns with my unwavering passion for machine learning research and my 
unwavering commitment to driving innovation in the realm of natural language processing (NLP).  
My profound journey into the intricate world of machine learning commenced in colle ge when I was 
oscillating and e xploring with a lot of  domains , a journey that has been shaped by a r elentless curiosity 
to unravel the profound intricacies of human language and the profound challenges it presents. One of 
the aspects of machine learning that deeply resonated with me was its interdisciplinary nature. I found 
that it seamlessly integrates mathematics, statistics, computer science, and domain -specific knowledge.


In [30]:
type(question)

str

In [31]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,pipeline
from langchain import HuggingFacePipeline

tokenizer = AutoTokenizer.from_pretrained("/home/asyin/llama/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("/home/asyin/llama/flan-t5-large")
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
llm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs={"temperature": 0, "max_length": 512},
)

In [32]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [33]:
print(QA_CHAIN_PROMPT)

input_variables=['context', 'question'] output_parser=None partial_variables={} template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. \n{context}\nQuestion: {question}\nHelpful Answer:" template_format='f-string' validate_template=True


In [34]:
print(question)

What are her interests


In [35]:
from langchain.chains import RetrievalQA 
qa_chain = RetrievalQA.from_chain_type(   
  llm=llm,   
  chain_type="stuff",   
  retriever=db.as_retriever(),   
  chain_type_kwargs={"prompt": QA_CHAIN_PROMPT} 
) 

#print(qa_chain)
result = qa_chain ({"query" : question}) 
#result=qa_chain(question)
print(result["result"])                            

Token indices sequence length is longer than the specified maximum sequence length for this model (585 > 512). Running this sequence through the model will result in indexing errors


machine learning research and her unwavering commitment to driving innovation in the realm of natural language processing
